In [4]:
import numpy as np
import pandas as pd

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [6]:
text = """ DENTAL HYGIENE: INTRODUCTION
 TO EMBEDDED SECURITY
 The sheer variety of embedded devices
 makes studying them fascinating, but
 that same variety can also leave you
 scratching your head over yet another
 shape, package, or weird integrated
 circuit (IC) and what it means in relation to its
 security. This chapter begins with a look at various
 hardware components and the types of software
 running on them. We then discuss attackers, various
 attacks, assets and security objectives, and
 countermeasures to provide an overview of how
 security threats are modeled. We describe the basics
 of creating an attack tree you can use both for
 defensive purposes (to find opportunities for
 countermeasures) and offensive purposes (to reason
 about the easiest possible attack). Finally, we
conclude with thoughts on coordinated disclosure in
 the hardware world.
 Hardware Components
 Let’s start by looking at the relevant parts of the physical
 implementation of an embedded device that you’re likely to encounter.
 We’l touch on the main bits you’l observe when first opening a
 device.
 Inside an embedded device is a printed circuit board (PCB) that
 generaly includes the fo lowing hardware components: processor,
 volatile memory, nonvolatile memory, analog components, and
 external interfaces (see
Figure 1-1).
Figure 1-1: Typical PCB of an embedded device
 The magic of computation happens in a processor (central
 processing unit, or CPU). In
Figure 1-1, the processor is embedded
 inside the System-on-Chip (SoC) in the center 1. Generaly, the
 processor executes the main software and operating system (OS),
 and the SoC contains additional hardware peripherals.
 Usualy implemented in dynamic RAM (DRAM) chips in discrete
 packages, volatile memory 2 is memory that the processor uses
 while it’s in action; its contents are lost when the device powers
down. DRAM memory operates at frequencies close to the processor
 frequency, and it needs wide buses in order to keep up with the
 processor.
 In
Figure 1-1, nonvolatile memory 3 is where the embedded
 device stores data that needs to persist after power to the device is
 removed. This memory storage can be in the form of EEPROMs,
 flash memory, or even SD cards and hard drives. Nonvolatile memory
 usualy contains code for booting as wel as stored applications and
 saved data.
 Although not very interesting for security in their own right, the
 analog components, such as resistors, capacitors, and inductors, are
 the starting point for side-channel analysis and fault-injection
 attacks, which we’l discuss at length in this book. On a typical PCB
 the analog components are al the little black, brown, and blue parts
 that don’t look like a chip and may have labels starting with “C,” “R,”
 or “L.”
 External interfaces provide the means for the SoC to make
 connections to the outside world. The interfaces can be connected to
 other commercial off-the-shelf (COTS) chips as part of the PCB
 system interconnect. This includes, for example, a high-speed bus
 interface to DRAM or to flash chips as wel as low-speed interfaces,
 such as I2C and SPI to a sensor. The external interfaces can also be
 exposed as connectors and pin headers on the PCB; for example,
 USB and PCI Express (PCIe) are examples of high-speed interfaces
 that connect devices externa ly. This is where a l communication
 happens; for example, with the internet, local debugging interfaces, or
 sensors and actuators. (See Chapter 2 for more details on interfacing
 with devices.)
 Miniaturization a lows an SoC to have more intellectual property
 (IP) blocks.
Figure 1-2 shows an example of an Intel Skylake SoC.
Figure 1-2: Intel Skylake SoC (public domain by Fritzchens Fritz)
 This die contains multiple cores, including the main central
 processing unit (CPU) cores, the Intel Converged Security and
 Management Engine (CSME), the graphics processing unit (GPU),
 and much more. Internal buses in an SoC are harder to access than
 external buses, making SoCs an inconvenient starting point for
 hacking. SoCs can contain the folowing IP blocks:
 Several (micro)processors and peripherals
 For instance, an application processor, a crypto engine, a video
 accelerator, and the I2C interface driver.
 Volatile memory
In the form of DRAM ICs stacked on top of the SoC, SRAMs, or
 register banks.
 Nonvolatile memory
 In the form of on-die read-only memory (ROM), one-time
programmable (OTP) fuses, EEPROM, and flash memory. OTP
 fuses typica ly encode critical chip configuration data, such as
 identity information, lifecycle stage, and anti-ro lback versioning
 information.
 Internal buses
 Though technicaly just a bunch of microscopic wires, the
 interconnect between the different components in the SoC is, in
 fact, a major security consideration. Think of this interconnect as
 the network between two nodes in an SoC. Being a network, the
 internal buses could be susceptible to spoofing, sniffing, injection,
 and al other forms of man-in-the-middle attacks. Advanced SoCs
 include access control at various levels to ensure that components
 in the SoC are “firewaled” from each other.
 Each of these components is part of the attack surface, the starting
 point for an attacker, and is therefore of interest. In Chapter 2, we’l
 study these external interfaces more in depth, and in Chapter 3, we’l
 look at ways to find information on the various chips and components.
 Software Components
 Software is a structured colection of CPU instructions and data that a
 processor executes. For our purposes, it doesn’t matter whether that
 software is stored in ROM, flash, or on an SD card—although it may
 come as a disappointment to our elder readers that we wil not cover
 punch cards. Embedded devices can contain some (or none) of the
 fo lowing types of software.
NOTE
 Although this book focuses on hardware attacks, often a
 hardware attack is used to compromise software. Via
 hardware vulnerabilities, attackers can gain access to parts of
 the software that are normally hard to access or that shouldn’t
 be accessible at all.
 Initial Boot Code
 The initial boot code is the set of instructions a processor executes
 when it’s first powered on. The initial boot code is generated by the
 processor manufacturer and stored in ROM. The main function of
 boot ROM code is to prepare the main processor to run the code that
 fo lows. Normaly, it alows a bootloader to execute in the field,
 including routines for authenticating a bootloader or for supporting
 alternate bootloader sources (such as through USB). It’s also used
 for support during manufacturing for personalization, failure analysis,
 debugging, and self-tests. Often the features available in the boot
 ROM are configured via fuses, which are one-time programmable
 bits integrated into the silicon that provide the option to disable some
 of the boot ROM functionality permanently when the processor leaves
 the manufacturing facility.
 Boot ROM has properties differentiating it from regular code: it is
 immutable, it is the first code to run on a system, and it must have
 access to the complete CPU/SoC to support manufacturing,
 debugging, and chip failure analysis. Developing ROM code requires
 a lot of care. Because it’s immutable, it’s usua ly not possible to patch
 a vulnerability in ROM that is detected post-manufacture (although
 some chips support ROM patching via fuses). Boot ROM executes
 before any network functionality is active, so physical access is
 required to exploit any vulnerabilities. A vulnerability exploited during
 this phase of boot likely results in direct access to the entire system.
Considering the high stakes for manufacturers in terms of reliability
 and reputation, in general, boot ROM code is usualy smal, clean,
 and wel verified (at least it should be).
 Bootloader
 The bootloader initializes the system after the boot ROM executes. It
 is typica ly stored on nonvolatile but mutable storage, so it can be
 updated in the field. The PCB’s original equipment manufacturer
 (OEM) generates the bootloader, alowing it to initialize PCB-level
 components. It may also optionaly lock down some security features
 in addition to its primary task of loading and authenticating an
 operating system or trusted execution environment (TEE). In
 addition, the bootloader may provide functionality for provisioning a
 device or debugging. Being the earliest mutable code to run on a
 device, the bootloader is an attractive target to attack. Less-secure
 devices may have a boot ROM that doesn’t authenticate the
 bootloader, a lowing attackers to replace the bootloader code easily.
 Bootloaders are authenticated with digital signatures, which are
 typica ly verified by embedding a public key (or the hash of a public
 key) in the boot ROM or fuses. Because this public key is hard to
 modify, it’s considered the root of trust. The manufacturer signs the
 bootloader using the private key associated with the public key, so
 the boot ROM code can verify and trust that the manufacturer
 produced it. Once the bootloader is trusted, it can, in turn, embed a
 public key for the next stage of code and provide trust that the next
 stage is authentic. This chain of trust can extend al the way down to
 applications running on an OS (see
Figure 1-3).
Figure 1-3: Chain of trust—bootloader stages and verification
 Theoretica ly, creating this chain of trust seems pretty secure, but
 the scheme is vulnerable to a number of attacks, ranging from
 exploiting verification weaknesses to fault injection, timing attacks,
 and more. See Jasper’s talk at Hardwear.io USA 2019 “Top 10
 Secure Boot Mistakes” on YouTube
 (
 https://www.youtube.com/watch?v=B9J8qjuxysQ/) for an overview
 of the top 10 mistakes.
 Trusted Execution Environment OS and Trusted
 Applications
 At the time of writing, the TEE is a rare feature in smaler embedded
 devices, but it’s very common in phones and tablets based on
 systems such as Android. The idea is to create a “virtual” secure SoC
 by partitioning an entire SoC into “secure” and “nonsecure” worlds.
 This means that every component on the SoC is either exclusively
 active in the secure world, exclusively active in the nonsecure world,
 or is able to switch between the two dynamicaly. For instance, an
 SoC developer may choose to put a crypto engine in the secure
 world, networking hardware in the nonsecure world, and alow the
 main processor to switch between the two worlds. This could alow
the system to encrypt network packets in the secure world and then
 transmit them via the nonsecure world—that is, the “normal world”—
 ensuring that the encryption key never reaches the main OS or a user
 application on the processor.
 On mobile phones and tablets, the TEE includes its own operating
 system, with access to al secure world components. The rich
 execution environment (REE) includes the “normal world” operating
 system, such as a Linux or iOS kernel and user applications.
 The goal is to keep al nonsecure and complex operations, such as
 user applications, in the nonsecure world, and al secure operations,
 such as banking applications, in the secure world. These secure
 applications are ca led trusted applications (TAs) . The TEE kernel is
 an attack target that, once compromised, typicaly provides complete
 access to both the secure and nonsecure worlds.
 Firmware Images
 Firmware is the low-level software that runs on CPUs or peripherals.
 Simple peripherals in a device are often fuly hardware based, but
 more complex peripherals can contain a microcontroler that runs
 firmware. For instance, most Wi-Fi chips require a firmware “blob” to
 be loaded after power-up. For those running Linux, a look at
 /lib/firmware shows how much firmware is involved in running PC
 peripherals. As with any piece of software, firmware can be complex
 and therefore sensitive to attacks.
 Main Operating System Kernel and Applications
 The main OS in an embedded system can be a general-purpose OS,
 like Linux, or a real-time OS, like VxWorks or FreeRTOS. Smart
 cards may contain proprietary OSs that run applications written in
 Java Card. These OSs can offer security functionality (for example,
 cryptographic services) and implement process isolation, which
means if one process is compromised, another process may stil be
 secure.
 An OS makes life easier for software developers who can rely on a
 broad range of existing functionality, but that may not be a viable
 option for smaler devices. Very smal devices may have no OS kernel
 but run only one bare-metal program to manage them. This usualy
 implies no process isolation, so compromising one function leads to
 compromising the entire device.
 Hardware Threat Modeling
 Threat modeling is one of the more important necessities in the
 defense of any system. Resources for defending a system are not
 unlimited, so analyzing how those resources are best spent to
 minimize attack opportunities is essential. This is the road to “good
 enough” security.
 When performing threat modeling, we roughly do the folowing:
 take a defensive view to identify the system’s important assets and
 ask ourselves how those assets should be secured. On the flip side,
 from an offensive viewpoint, we could identify who the attackers
 might be, what their goals might be, and what attacks they could
 choose to attempt. These considerations provide insights into what to
 protect and how to protect the most valuable assets.
 The standard reference work for threat modeling is Adam
 Shostack’s book Threat Modeling: Designing for Security (Wiley,
 2014). The broad field of threat modeling is fascinating, as it includes
 security of the development environment through to manufacturing,
 supply chain, shipping, and the operational lifetime. We’l address the
 basic aspects of threat modeling here and apply them to embedded
 device security, focusing on the device itself.
 What Is Security?
The Oxford English Dictionary defines security as “the state of being
 free from danger or threat.” This rather binary definition implies that
 the only secure system is either one that no one would bother to
 attack or one that can defend every threat. The former, we cal a
 brick, because it no longer can boot; the latter, we cal a unicorn,
 because unicorns don’t exist. There is no perfect security, so you
 could argue that any defense is not worth the effort. This attitude is
 known as security nihilism. However, that attitude disregards the
 important fact that a cost-benefit trade-off is associated with each
 and every attack.
 We al understand cost and benefit in terms of money. For an
 attacker, costs are usua ly related to buying or renting equipment
 needed for carrying out attacks. Benefits come in the form of
 fraudulent purchases, stolen cars, ransomware payouts, and slot
 machine cash-outs, just to name a few.
 The costs and benefits of performing attacks are not exclusively
 monetary, however. An obvious non-monetary cost is time; a less
 obvious cost is attacker frustration. For example, an attacker who is
 hacking for fun may simply move on to another target in the face of
 frustration. There is surely a defense lesson here. See Chris Domas’s
 talk at DEF CON 23 for more on this idea: “Repsych: Psychological
 Warfare in Reverse Engineering.” Nonmonetary benefits include
 gathering personaly identifiable information and fame derived from
 conference publications or successful sabotage (although those
 benefits may also be monetized).
 In this book, we consider a system “secure enough” if the cost of
 an attack is higher than the benefit. A system design may not be
 impenetrable, but it should be hard enough that no one wil see an
 entire attack through to success. In summary, threat modeling is the
 process of determining how to reach a secure-enough state in a
 particular device or system. Next, let’s look at several aspects that
 affect the benefits and costs of an attack.
 Attacks Through Time
The US National Security Agency (NSA) has a saying: “Attacks
 always get better; they never get worse.” In other words, attacks get
 cheaper and stronger over time. This tenet particularly holds at larger
 timescales, because of increased public knowledge of a target,
 decreased cost of computing power, and the ready availability of
 hacking hardware. The time from a chip’s initial design to final
 production can span several years, folowed by at least a year to
 implement the chip in a device, resulting in three to five years before
 it’s operational in a commercial environment. This chip may need to
 remain operational for a few years (in the case of Internet of Things
 [IoT] products), or 10 years (for an electronic passport), or even for
 20 years (in automotive and medical environments). Thus, designers
 need to take into account whatever attacks might be happening 5 to
 25 years hence. This is clearly impossible, so often software fixes
 have to be pushed out to mitigate unpatchable hardware problems.
 To put it in perspective, 25 years ago a smart card may have been
 very hard to break, but after working your way through this book, a
 25-year-old smart card should pose little resistance in extracting its
 keys.
 Cost differences also appear on smaler timescales when going
 from an initial attack to repeating that attack. The identification phase
 involves identifying vulnerabilities. The exploitation phase fo lows,
 which involves using the identified vulnerabilities to exploit a target. In
 the case of (scalable) software vulnerabilities, the identification cost
 may be significant, but the exploitation cost is almost zero, as the
 attack can be automated. For hardware attacks, the exploitation cost
 may stil be significant.
 On the benefits side, attacks typica ly have a limited window within
 which they have value. Cracking Commodore 64 copy protection
 today provides little monetary advantage. A video stream of your
 favorite sportsba l game has high value only while the game is in
 progress and before the result is known. The day afterward, its value
 is significantly lower.
Scalability of Attacks
 The identification and exploitation phases of software and hardware
 attacks differ significantly from each other in terms of cost and
 benefit. The cost of the hardware exploitation phase may be
 comparable to that of the identification phase, which is uncommon for
 software. For instance, a securely designed smart card payment
 system makes use of diversified keys so that finding the key on one
 card means you learn nothing about the key of another card. If card
 security is sufficiently strong, attackers need weeks or months and
 expensive equipment to make a few thousand dolars’ worth of
 fraudulent purchases on each card. They must repeat the process for
 every new card to gain the next few thousand dolars. If the cards are
 that strong, obviously no business case exists for financia ly motivated
 attackers; such an attack scales poorly.
 On the other hand, consider the Xbox 360 modchips.
Figure 1-4
 shows the Xenium ICE modchip as the white PCB to the left.
Figure 1-4: Xenium ICE modchip in an Xbox, used to bypass code verification
 (photo by Helohe, CC BY 2.5 license)
 A Xenium ICE modchip on the left in
Figure 1-4 is soldered to the
 main Xbox PCB in order to perform its attack. The board automates
 a fault injection attack to load arbitrary firmware. This hardware
 attack is so easily performed, se ling modchips could be turned into a
 business; therefore, we say it “scales wel” (Chapter 13 provides a
 more detailed description of this attack).
 Hardware attackers benefit from economies of scale, but only if the
 exploitation cost is very low. One example of this is hardware attacks
 to extract secrets that can then be used at scale, such as recovery of
 a master firmware update key hidden in hardware facilitating access
to a multitude of firmware. Another example is the once-off operation
 of extracting boot ROM or firmware code, which can expose system
 vulnerabilities that can be exploited many times over.
 Fina ly, scale is not important for some hardware attacks. For
 example, hacking once would be sufficient for obtaining an
 unencrypted copy of a video from a digital rights management (DRM)
 system that is then pirated, as is the case with launching a single
 nuclear missile or decrypting a president’s tax returns.
 The Attack Tree
 An attack tree visualizes the steps an attacker takes when going from
 the attack surface to the ability to compromise an asset, alowing us
 to analyze an attack strategy systematica ly. The four ingredients we
 consider in an attack tree are attackers, attacks, assets (security
 objectives), and countermeasures (see
Figure 1-5).
 Figure 1-5: Relationship between elements of threat modeling
 Profiling the Attackers
 Profiling the attackers is important because attackers have motives,
 resources, and limitations. You could claim that botnets or worms are
 nonhuman players lacking motivation, but a worm is initia ly launched
 by a person pressing ENTER with glee, anger, or greedy anticipation.
 """

In [7]:
tokenizer = Tokenizer()

In [8]:
tokenizer.fit_on_texts([text])

In [9]:
tokenizer.word_index

{'the': 1,
 'to': 2,
 'a': 3,
 'of': 4,
 'and': 5,
 'in': 6,
 'is': 7,
 'for': 8,
 'that': 9,
 'an': 10,
 'on': 11,
 'or': 12,
 'be': 13,
 'this': 14,
 'attack': 15,
 'can': 16,
 'as': 17,
 'it': 18,
 'hardware': 19,
 'are': 20,
 'system': 21,
 'security': 22,
 'attacks': 23,
 '1': 24,
 'may': 25,
 'rom': 26,
 'boot': 27,
 'at': 28,
 'soc': 29,
 'code': 30,
 'software': 31,
 'device': 32,
 'processor': 33,
 'secure': 34,
 'with': 35,
 'components': 36,
 'memory': 37,
 'figure': 38,
 'cost': 39,
 'but': 40,
 'we': 41,
 'ly': 42,
 'one': 43,
 'from': 44,
 'bootloader': 45,
 'embedded': 46,
 'attackers': 47,
 'firmware': 48,
 'threat': 49,
 'world': 50,
 'main': 51,
 'applications': 52,
 'such': 53,
 'have': 54,
 'access': 55,
 'key': 56,
 'by': 57,
 'os': 58,
 'not': 59,
 'example': 60,
 'more': 61,
 'so': 62,
 'card': 63,
 'modeling': 64,
 'devices': 65,
 'pcb': 66,
 'interfaces': 67,
 'it’s': 68,
 'which': 69,
 'time': 70,
 'its': 71,
 'when': 72,
 'see': 73,
 'al': 74,
 'public': 75,


In [10]:
input_sequences = []
for sentence in text.split('\n'):
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

  for i in range(1, len(tokenized_sentence)):
    input_sequences.append(tokenized_sentence[:i+1])

In [11]:
print(input_sequences)

[[415, 416], [415, 416, 417], [2, 46], [2, 46, 22], [1, 418], [1, 418, 252], [1, 418, 252, 4], [1, 418, 252, 4, 46], [1, 418, 252, 4, 46, 65], [179, 419], [179, 419, 94], [179, 419, 94, 253], [179, 419, 94, 253, 40], [9, 420], [9, 420, 252], [9, 420, 252, 16], [9, 420, 252, 16, 79], [9, 420, 252, 16, 79, 421], [9, 420, 252, 16, 79, 421, 95], [422, 180], [422, 180, 423], [422, 180, 423, 181], [422, 180, 423, 181, 424], [422, 180, 423, 181, 424, 96], [425, 426], [425, 426, 12], [425, 426, 12, 427], [425, 426, 12, 427, 254], [255, 428], [255, 428, 5], [255, 428, 5, 97], [255, 428, 5, 97, 18], [255, 428, 5, 97, 18, 98], [255, 428, 5, 97, 18, 98, 6], [255, 428, 5, 97, 18, 98, 6, 429], [255, 428, 5, 97, 18, 98, 6, 429, 2], [255, 428, 5, 97, 18, 98, 6, 429, 2, 71], [22, 14], [22, 14, 99], [22, 14, 99, 430], [22, 14, 99, 430, 35], [22, 14, 99, 430, 35, 3], [22, 14, 99, 430, 35, 3, 100], [22, 14, 99, 430, 35, 3, 100, 28], [22, 14, 99, 430, 35, 3, 100, 28, 133], [19, 36], [19, 36, 5], [19, 36, 5

In [12]:
max_len = max([len(x) for x in input_sequences])

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding = 'pre')

In [14]:
x = padded_input_sequences[:, :-1]
y = padded_input_sequences[:, -1]

In [15]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes = len(tokenizer.word_index) + 1)

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

In [28]:
model = Sequential()
model.add(Embedding(1079, 100, input_length=max_len, input_shape=(max_len,)))
model.add(LSTM(150))
model.add(Dense(1079, activation = 'softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [34]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [35]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (None, 16, 100)             │         107,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 150)                 │         150,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1079)                │         162,929 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 421,429 (1.61 MB)

 Trainable params: 421,429 (1.61 MB)

 Non-trainable params: 0 (0.00 B)

In [36]:
model.fit(x, y, epochs = 100)

Epoch 1/100
96/96 ━━━━━━━━━━━━━━━━━━━━ 8s 55ms/step - accuracy: 0.0517 - loss: 6.6550
Epoch 2/100
96/96 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - accuracy: 0.0570 - loss: 5.9803
Epoch 3/100
96/96 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.0566 - loss: 5.9069
Epoch 4/100
96/96 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.0528 - loss: 5.7795
Epoch 5/100
96/96 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.0648 - loss: 5.7327
Epoch 6/100
96/96 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - accuracy: 0.0628 - loss: 5.6162
Epoch 7/100
96/96 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.0716 - loss: 5.5048
Epoch 8/100
96/96 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 0.0986 - loss: 5.2189
Epoch 9/100
96/96 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - accuracy: 0.0983 - loss: 5.1354
Epoch 10/100
96/96 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.1377 - loss: 4.8583
Epoch 11/100
96/96 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.1446 - loss: 4.7415
Epoch 12/100
96/96 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step

In [50]:
input = "nice"

for i in range(10):
  #tokenize
  token_text = tokenizer.texts_to_sequences([input])[0]
  #padding
  padded_token_text = pad_sequences([token_text], maxlen = max_len, padding = 'pre')
  #predict
  pos = np.argmax(model.predict(padded_token_text))

  for word, index in tokenizer.word_index.items():
    if index == pos:
      input += " " + word
      print(input)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
nice identifying
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
nice identifying vulnerabilities
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
nice identifying vulnerabilities attackers
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
nice identifying vulnerabilities attackers can
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
nice identifying vulnerabilities attackers can gain
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
nice identifying vulnerabilities attackers can gain access
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
nice identifying vulnerabilities attackers can gain access to
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
nice identifying vulnerabilities attackers can gain access to parts
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
nice identifying vulnerabilities attackers can gain access to parts of
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
nice identifying vulnerabilities attackers can gain access to parts of the
